In [1]:
# Importing dependencies
import os
import pandas as pd
from pandas.io.json import json_normalize
import numpy as np
import pprint

In [2]:
# Path to source JSON
businessJson=os.path.join('sourceData', 'business.json')

In [3]:
# Path to Yelp food/restaurant categories csv
yelpCategories=os.path.join('sourceData', 'yelpCategories.csv')

In [4]:
# Creating pd dataframe
business_raw=pd.read_json(businessJson, lines=True)

In [5]:
# Select only the businesses in Ontario
business_on=business_raw.loc[business_raw['state'] == 'ON']

In [6]:
# Dropping any rows with blank values in these categories
business=business_on.dropna(subset=['name', 'address', 'postal_code', 'city', 'state', 'latitude', 'longitude', 'attributes',
                                               'categories', 'hours']).reset_index(drop=True)

In [7]:
# Regex to fix spelling mistakes 
business=business.replace({'city': {'^AGINCOURT$': 'Agincourt',
                                                   '^Bradford West Gwillimbury$': 'Bradford',
                                                   '^East Ajax$': 'Ajax',
                                                   '^Caledon.{,8}$': 'Caledon',
                                                   '^East Gwil{1,2}imbury$': 'East Gwillimbury',
                                                   '(?i)^.*icoke$': 'Etobicoke',
                                                   '^.{,9}Toro?nto.{,9}$': 'Toronto',
                                                   'Malton': 'Mississauga',
                                                   '^.{,5}Missis{1,2}a?ua?g.{1,2}$': 'Mississauga',
                                                   '^Regional Municipality of York$': 'North York',
                                                   '(?i)^North.{0,2}York$': 'North York',
                                                   '^York Regional Municipality$': 'York',
                                                   '^Willowdale$': 'North York',
                                                   '^North of Brampton$': 'Brampton',
                                                   '(?i)^Oak.?ridges$': 'Oak Ridges',
                                                   '^oakville$': 'Oakville',
                                                   '(?i)^Richmond?.?Hill?$': 'Richmond Hill',
                                                   '^.{,8}Scar.?bo?rough$': 'Scarborough',
                                                   '^.{,11}Stouffville$': 'Stouffville',
                                                   '(?i)^Thornhil{,2}$': 'Thornhill',
                                                   '^.*Vaugh.{,3}$': 'Vaughan',
                                                   '^Wh.?i.?by$': 'Whitby'}}, regex=True)

In [8]:
# Only taking these columns
business=business.loc[:, ['name', 'address', 'postal_code', 'city', 'state', 'latitude', 'longitude','categories', 'stars', 'hours', 'attributes']]
business.columns=['Name', 'Address', 'Postal_code', 'City', 'Province', 'Latitude', 'Longitude', 'Categories', 'Stars', 'Hours', 'Attributes']

In [9]:
# Turning the hours column in to a df
hours_raw=json_normalize(data=business['Hours'])
business.drop(columns='Hours', inplace=True)
# hours_raw.head()

,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
0,9:0-1:0,9:0-0:0,9:0-1:0,9:0-0:0,9:0-0:0,9:0-0:0,9:0-0:0
1,9:0-19:0,9:0-19:0,10:0-16:0,NaN,9:0-20:0,9:0-20:0,9:0-20:0
2,16:0-21:30,16:0-23:0,15:30-17:30,17:30-19:45,16:0-23:0,16:0-23:0,16:0-23:0
3,6:0-22:0,6:0-22:0,8:0-18:0,9:0-18:0,6:0-22:0,6:0-22:0,6:0-22:0
4,9:0-20:0,9:0-20:0,9:0-20:0,9:0-20:0,9:0-20:0,9:0-20:0,9:0-20:0


## Handling the hours column

In [10]:
# Reorganise columns
hours_raw=hours_raw.loc[:,['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']]

In [11]:
# Create a new df with opening and closing hours
columns=hours_raw.columns
hours=hours_raw

In [12]:
# Loop through and split the columns
for column in columns:
    hours[[f"{column}_open", f"{column}_close"]]=hours_raw[column].str.split('-', expand=True)
hours.drop(columns=columns, inplace=True)
hours=hours.apply(lambda x: x.str.strip())

In [13]:
# Add hours column to the original DF
business=business.join(hours)
# restaurant.drop(columns='Hours', inplace=True)
# restaurant.tail()

## Setting up the dataframes


In [14]:
# Turning the Categories column in to a df and adding restaurant id
category = business['Categories'].str.split(',', expand=True)
category['Restaurant_id']=business.index
category=pd.melt(category, id_vars='Restaurant_id', value_name='Category').drop(columns='variable').sort_values('Restaurant_id')

In [15]:
# Turning the Attributes column in to a df and adding restaurant id
attributesRaw=json_normalize(data=business['Attributes'])
business.drop(columns='Attributes', inplace=True)
attributesRaw['Restaurant_id']=business.index

# Keeping only the relevant columns
attributes=attributesRaw.loc[:, ['Restaurant_id', 'Alcohol', 'Ambience', 'BestNights', 'BikeParking', 'BusinessAcceptsCreditCards', 'BusinessParking', 'ByAppointmentOnly', 'DietaryRestrictions',
                                 'DogsAllowed', 'GoodForKids', 'GoodForMeal', 'HappyHour', 'HasTV', 'Music', 'NoiseLevel', 'OutdoorSeating', 'RestaurantsAttire', 'RestaurantsGoodForGroups',
                                 'RestaurantsPriceRange2', 'RestaurantsReservations', 'RestaurantsTableService', 'RestaurantsTakeOut', 'Smoking', 'WheelchairAccessible', 'WiFi']]
attributes.head()

,Restaurant_id,Alcohol,Ambience,BestNights,BikeParking,BusinessAcceptsCreditCards,BusinessParking,ByAppointmentOnly,DietaryRestrictions,DogsAllowed,...,OutdoorSeating,RestaurantsAttire,RestaurantsGoodForGroups,RestaurantsPriceRange2,RestaurantsReservations,RestaurantsTableService,RestaurantsTakeOut,Smoking,WheelchairAccessible,WiFi
0,0,u'full_bar',"{'romantic': False, 'intimate': False, 'classy...",NaN,False,NaN,"{'garage': False, 'street': False, 'validated'...",NaN,NaN,NaN,...,False,u'casual',True,2,True,True,True,NaN,NaN,u'no'
1,1,NaN,NaN,NaN,NaN,NaN,"{'garage': False, 'street': False, 'validated'...",NaN,NaN,NaN,...,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN
2,2,NaN,NaN,NaN,True,NaN,"{'garage': False, 'street': False, 'validated'...",False,NaN,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN
3,3,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN
4,4,NaN,NaN,NaN,True,NaN,"{'garage': False, 'street': False, 'validated'...",NaN,NaN,NaN,...,NaN,NaN,NaN,2,NaN,NaN,True,NaN,NaN,NaN


In [21]:
# Turning the Ambience column in to a df and adding restaurant id
ambienceRaw=json_normalize(data=attributes['Ambience'], )
attributes.drop('Ambience', inplace=True)
ambienceRaw['Restaurant_id']=business.index
ambienceRaw=ambienceRaw.apply(lambda x: x.str.strip())
ambienceRaw.head()

AttributeError: 'str' object has no attribute 'values'

In [ ]:
# Turning the Parking column in to a df and adding restaurant id
parking=json_normalize(data=attributes['BusinessParking'])
attributes.drop('BusinessParking', inplace=True)
parking['Restaurant_id']=business.index
parking=parking.apply(lambda x: x.str.strip())
parking.head()

In [ ]:
# Turning the GoodForMeal column in to a df and adding restaurant id
goodForMeal=json_normalize(data=attributes['GoodForMeal'])
attributes.drop('GoodForMeal', inplace=True)
goodForMeal['Restaurant_id']=business.index
goodForMeal=goodForMeal.apply(lambda x: x.str.strip())
goodForMeal.head()

In [ ]:
attributes.iloc[0]

## Handling the categories column

In [ ]:
# Eliminating None categories
category=category.loc[~category['Category'].isin([None])].reset_index(drop=True)
category['Category'] = category['Category'].str.strip()

In [ ]:
# Read csv with list of yelp categories
categories=pd.read_csv(yelpCategories)
categories['Category'] = categories['Category'].str.strip()

In [ ]:
# Take only the categories from the list of yelp categories
categoryFiltered=category.loc[category['Category'].isin(categories['Category'])]
# categoryFiltered.tail()

In [ ]:
# Creating a list of unique food categories
uniqueCategories=categoryFiltered['Category'].unique()
uniqueCategories.sort()
# print(uniqueCategories)

In [ ]:
# Filter out categories that don't apply
categoryFiltered=categoryFiltered.loc[category['Category'].isin(['Acai Bowls', 'Afghan', 'African', 'Arabian', 'Argentine', 'Asian Fusion', 'Australian', 'Austrian', 'Bagels', 'Bangladeshi',
                                                                   'Barbeque', 'Basque', 'Beer Bar', 'Beer Gardens', 'Beer Hall', 'Belgian', 'Bistros', 'Brasseries', 'Brazilian', 'Breakfast & Brunch',
                                                                   'Brewpubs', 'British', 'Bubble Tea', 'Buffets', 'Burgers', 'Burmese', 'Cafes', 'Cajun/Creole', 'Cambodian', 'Cantonese', 'Caribbean',
                                                                   'Champagne Bars', 'Cheese Shops', 'Cheesesteaks', 'Chicken Shop', 'Chicken Wings', 'Chinese', 'Chocolatiers & Shops',
                                                                   'Cocktail Bars', 'Coffee & Tea', 'Colombian', 'Comfort Food', 'Creperies', 'Cuban', 'Cupcakes',
                                                                   'Czech', 'Czech/Slovakian', 'Delicatessen', 'Delis', 'Desserts', 'Dim Sum', 'Diners', 'Dive Bars', 'Do-It-Yourself Food', 'Donairs',
                                                                   'Egyptian', 'Ethical Grocery', 'Ethiopian', 'Falafel', 'Fast Food', 'Filipino', 'Fish & Chips', 'Fondue',
                                                                   'Food Stands', 'Food Trucks', 'French', 'Fruits & Veggies', 'Gastropubs', 'Gelato', 'German', 'Gluten-Free', 'Greek', 'Hainan', 'Haitian',
                                                                   'Hakka', 'Halal', 'Hawaiian', 'Himalayan/Nepalese', 'Hong Kong Style Cafe', 'Hot Dogs', 'Hot Pot', 'Hungarian', 'Iberian',
                                                                   'Ice Cream & Frozen Yogurt', 'Imported Food', 'Indian', 'Indonesian', 'International', 'Internet Cafes', 'Irish', 'Irish Pub', 'Italian',
                                                                   'Izakaya', 'Japanese', 'Japanese Sweets', 'Juice Bars & Smoothies', 'Kebab', 'Kombucha', 'Korean', 'Kosher', 'Laotian', 'Latin American',
                                                                   'Lebanese', 'Live/Raw Food', 'Lounges', 'Macarons', 'Malaysian', 'Mauritius', 'Mediterranean', 'Mexican', 'Middle Eastern',
                                                                   'Milkshake Bars', 'Minho', 'Modern European', 'Mongolian', 'Moroccan', 'Nicaraguan', 'Noodles', 'Pakistani', 'Pan Asian', 'Pasta Shops',
                                                                   'Persian/Iranian', 'Peruvian', 'Pizza', 'Poke', 'Polish', 'Portuguese',
                                                                   'Poutineries', 'Pubs', 'Ramen', 'Reunion', 'Russian', 'Salad', 'Salvadoran', 'Sandwiches', 'Scandinavian', 'Scottish', 'Seafood',
                                                                   'Seafood Markets', 'Shanghainese', 'Shaved Ice', 'Shaved Snow', 'Singaporean', 'Slovakian', 'Smokehouse', 'Soul Food', 'Soup',
                                                                   'South African', 'Southern', 'Spanish', 'Specialty Food', 'Sports Bars', 'Sri Lankan', 'Steakhouses', 'Street Vendors', 'Supper Clubs',
                                                                   'Sushi Bars', 'Swiss Food', 'Syrian', 'Szechuan', 'Tacos', 'Taiwanese', 'Tapas Bars', 'Tapas/Small Plates', 'Tea Rooms', 'Tempura',
                                                                   'Teppanyaki', 'Tex-Mex', 'Thai', 'Themed Cafes', 'Tiki Bars', 'Tonkatsu', 'Trinidadian', 'Turkish', 'Udon', 'Ukrainian', 'Vegan',
                                                                   'Vegetarian', 'Venezuelan', 'Vietnamese', 'Waffles', 'Whiskey Bars', 'Wine Bars', 'Wraps'])]

In [ ]:
categoryFiltered.count()

In [ ]:
# Creating a list of updated unique food categories
uniqueCategories=categoryFiltered['Category'].unique()
uniqueCategories.sort()
# print(uniqueCategories)

In [ ]:
business=business.loc[restaurant.index.isin(categoryFiltered['Restaurant_id'])]
# restaurant.tail()

# TODO add the category values to the master table with with a loop

## Handling the attributes df

In [ ]:
attributesRaw.head()

In [ ]:
attributes['RestaurantsCounterService'].unique()

In [ ]:
# Filter by Appointment false

In [ ]:
# Keeping only the relevant columns
attributes=attributesRaw.loc[:, ['Alcohol', 'Ambience', 'BestNights', 'BikeParking', 'BusinessAcceptsCreditCards', 'BusinessParking', 'ByAppointmentOnly', 'DietaryRestrictions', 'DogsAllowed',
                                 'GoodForKids', 'GoodForMeal', 'HappyHour', 'HasTV', 'Music', 'NoiseLevel', 'OutdoorSeating', 'RestaurantsAttire', 'RestaurantsGoodForGroups', 'RestaurantsPriceRange2',
                                 'RestaurantsReservations', 'RestaurantsTableService', 'RestaurantsTakeOut', 'Smoking', 'WheelchairAccessible', 'WiFi']]